# Mygea (All-in-One version)

Merupakan versi gea berupa satu file besar kumpulan keseluruhan source code mygea.


## Algoritma genetika

Tugas Pengantar Kecerdasan Buatan

ditulis oleh Jordi Yaputra (130110353)

lisensi: unlicense

## The Unlicense

This is free and unencumbered software released into the public domain.

Anyone is free to copy, modify, publish, use, compile, sell, or
distribute this software, either in source code form or as a compiled
binary, for any purpose, commercial or non-commercial, and by any
means.

In jurisdictions that recognize copyright laws, the author or authors
of this software dedicate any and all copyright interest in the
software to the public domain. We make this dedication for the benefit
of the public at large and to the detriment of our heirs and
successors. We intend this dedication to be an overt act of
relinquishment in perpetuity of all present and future rights to this
software under copyright law.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS BE LIABLE FOR ANY CLAIM, DAMAGES OR
OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

For more information, please refer to <https://unlicense.org>

## Mygea

### import

In [ ]:
# tools plotting dan grafik
from matplotlib import pyplot as plt
# tools matematika
from math import sin, cos, floor
from random import random as rand
# untuk class Stop
import enum

### utils.py

Fungsi-fungsi umum untuk membantu komputasi gea

In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
  leftSpan = leftMax - leftMin
  rightSpan = rightMax - rightMin
  valueScaled = float(value - leftMin) / float(leftSpan)
  return rightMin + (valueScaled * rightSpan)

def toDecimal(bin_list):
  dec = 0
  l = len(bin_list) - 1
  for i, x in enumerate(bin_list):
    dec += pow(2, l - i) * x
  return dec

### gea.py

Modul inti untuk menghitung ga

In [ ]:
# %% enumerasi
class Stop(enum.Enum):
  MAX_IT = 0      # always on, stop when max iteration reached
  TRESHOLD = 1    # stop if fitness >= treshold value
  NO_IMPROVE = 3  # stop if no improvement for certain generation

In [ ]:
class Individu:
  def __init__(self,
                ranges:tuple=((0,1),(0,1)),
                resolusi:int=5,
                kromosom:list=None):

    assert type(ranges) is tuple
    for rg in ranges:
      assert len(rg) is 2

    self.kromosom = kromosom or \
        [round(rand()) for _ in range(resolusi * len(ranges))]
    self.res = resolusi
    self.ranges = ranges

  def getFenotip(self):
    l = self.res
    up = 2**l-1
    return tuple(translate(toDecimal(self.kromosom[l*i:l*(i+1)]),
                                      0, up, ran[0], ran[1]) \
                  for i, ran in enumerate(self.ranges))

In [ ]:
class Gea:
  def __init__(self,
               fungsi_fitness:callable,
               ranges:tuple,
               resolusi:int,
               ukuran_populasi=50):

    for rg in ranges:
      assert len(rg) is 2

    self.fungsi_fitness = fungsi_fitness
    self.resolusi = resolusi
    self.ranges = ranges
    self.ukuran_populasi = ukuran_populasi
    self.reset()

  def reset(self):
    self.best_individu = (None, 0)
    self.fitness = 0
    self.populasi = [Individu(self.ranges, self.resolusi)
                     for _ in range(self.ukuran_populasi)]

  def __hitungFitness(self):
    fitness_list = []
    for individu in self.populasi:
      fenotip = individu.getFenotip()
      fit = self.fungsi_fitness(fenotip)
      fitness_list.append(fit)
    return fitness_list

  def __hitungPeluang(self, fitness_list, verbose=False):
    total_fitness = sum(fitness_list)
    peluang_list = list(map(lambda x: x/total_fitness, fitness_list))
    return peluang_list

  def __printTabel(self, fitness_list, peluang_list, verbose=2):
    if verbose > 2:
      print('fen', 'fitness', 'peluang', 'kromosom', sep='\t')
      for individu, fit, peluang in zip(self.populasi,
                                        fitness_list,
                                        peluang_list):
        fen = individu.getFenotip()
        print(tuple('%.2f' % f for f in fen),
              '%.2f' % fit,
              '%.2f' % peluang,
              individu.kromosom,
              sep='\t')
    print('fitness: %.2f' % max(fitness_list))

  def __rouletteWheel(self, peluang_list):
    r = rand()
    batas = 0
    for idx, p in enumerate(peluang_list):
      batas += p
      if r < batas:
        return idx

  def __seleksiOrtu(self, peluang_list, banyak_pasangan):
    return [(self.__rouletteWheel(peluang_list),
            self.__rouletteWheel(peluang_list))
            for _ in range(banyak_pasangan)]

  def __pindahSilang(self, pasangan):
    tipot = floor(rand() * (len(pasangan[0].kromosom) - 1)) + 1
    k1 = pasangan[0].kromosom[:tipot] + pasangan[1].kromosom[tipot:]
    k2 = pasangan[1].kromosom[:tipot] + pasangan[0].kromosom[tipot:]
    return (k1, k2)

  def __mutasi(self, peluang_mutasi):
    for individu in self.populasi:
      if (rand() < peluang_mutasi):
        posisi = floor(rand() * len(individu.kromosom))
        individu.kromosom[posisi] = (individu.kromosom[posisi] + 1) % 2

  def __urutanPopulasi(self, populasi, urutan):
    urutan_list = sorted(range(len(urutan)), key=lambda i: urutan[i])
    urutan_list.reverse()
    populasi_terurut = [populasi[urutan] for urutan in urutan_list]
    return populasi_terurut

  def __routineFitPel(self, verbose):
    fitness_list = self.__hitungFitness()
    peluang_list = self.__hitungPeluang(fitness_list)
    best_fit = max(fitness_list)
    if best_fit > self.best_individu[1]:
      self.best_individu = (self.populasi[fitness_list.index(best_fit)],
                            best_fit)

    if verbose:
      if verbose is 1:
        print('*', end='')
      else :
        self.__printTabel(fitness_list, peluang_list, verbose)

    return (best_fit, peluang_list)

  def __routineRegenerasi(self, peluang_list, crossover_rate):
    banyak_pasangan = round((self.ukuran_populasi * crossover_rate) / 2)
    pasangan_index_ortu = self.__seleksiOrtu(peluang_list, banyak_pasangan)

    self.populasi = self.__urutanPopulasi(self.populasi, peluang_list)
    populasi_baru = []

    for pasangan_idx in pasangan_index_ortu:
      pasangan = tuple(self.populasi[idx] for idx in pasangan_idx)
      kromosom_list = self.__pindahSilang(pasangan)
      for kromosom in kromosom_list:
        populasi_baru.append(Individu(self.ranges,
                                      self.resolusi,
                                      kromosom))

    l = len(self.populasi) - len(populasi_baru)
    self.populasi[l:] = populasi_baru

  def fit(self,
          stopping_crit:tuple=(Stop.MAX_IT,),
          maks_generasi:int=200,
          crossover_rate:float=.5,
          peluang_mutasi:float=.03,
          rekam_history:bool=True,
          verbose=1):
    assert crossover_rate >= 0 and crossover_rate <= 1

    iterasi = 0
    fitness_history = []

    if verbose is 1:
      print('Progress: [', end='')

    best_fit, peluang_list = self.__routineFitPel(verbose)


    if rekam_history:
      fitness_history.append(best_fit)


    while iterasi < maks_generasi:

      # mulai mekanisme stopping
      if stopping_crit[0] == Stop.TRESHOLD:
        if best_fit >= stopping_crit[1]:
          # stop iterasi
          break

      elif stopping_crit[0] == Stop.NO_IMPROVE:
        l = len(fitness_history)
        if l > stopping_crit[1]:
          is_improving = False
          for i in range(stopping_crit[1]):
            if fitness_history[l-1-i] != fitness_history[l-i-2]:
              is_improving = True
              break
          if not is_improving:
            # stop iterasi
            break
      # akhir mekanisme stopping

      self.__routineRegenerasi(peluang_list, crossover_rate)
      self.__mutasi(peluang_mutasi)
      best_fit, peluang_list = self.__routineFitPel(verbose)

      if rekam_history:
        fitness_history.append(best_fit)

      iterasi += 1

    if verbose is 1:
      print(']')

    fenotip = self.best_individu[0].getFenotip()

    return (fenotip, iterasi, fitness_history)

## main.py

In [ ]:
# %% pengaturan
verbosity_level = 1  # rentang [0, 3]
rekam_history = True  # set True jika butuh graph
plt.style.use('seaborn-whitegrid')

In [ ]:
# %% fungsi hipothesis dan fitness
def h(fenotip):
  x1, x2 = fenotip
  return cos(x1) * sin(x2) - x1 / (x2**2 + 1)

def fungsiFitnessKurang(fenotip):
  return 3 - h(fenotip)

def fungsiFitnessBagi(fenotip):
  return 1 / (h(fenotip) + 2)

def fungsiFitnessEksponen(fenotip):
  return pow(4, -h(fenotip))

In [ ]:
# %% hyper parameter

""" tebakan terbaik
# hasil:
# rata-rata generasi = 9.9
# rata-rata fitness = 16.490743452040977

ukuran_populasi = 750
resolusi = 10
peluang_mutasi = .09
crossover_rate = .7
# """

""" hasil GA Tuner #1
# hasil:
# rata-rata generasi = 6.05
# rata-rata fitness  = 16.49107486703866
# selalu konvergen global

ukuran_populasi = 779
resolusi = 8
peluang_mutasi = 0.021176470588235297
crossover_rate = 0.8694117647058823
# """

""" hasil GA Tuner #2
# hasil:
# rata-rata generasi = 4.2
# rata-rata fitness  = 16.4905685130365
# selalu konvergen global

ukuran_populasi = 591
resolusi = 7
peluang_mutasi = 0.13592156862745097
crossover_rate = 0.9470588235294117
# """

# """ hasil GA Tuner #3
# hasil:
# rata-rata generasi = 3.4
# rata-rata fitness  = 16.4905685130365
# selalu konvergen global

ukuran_populasi = 797
resolusi = 7
peluang_mutasi = 0.16945098039215686
crossover_rate = 0.9400000000000001
# """

fungsiFitness = fungsiFitnessEksponen
maks_generasi = int((10000 - ukuran_populasi * (1 - crossover_rate)) \
                   / (ukuran_populasi * crossover_rate))
print('Maksimum generasi:', maks_generasi)

# stopping_crit = (gea.Stop.MAX_IT,)
# stopping_crit = (gea.Stop.TRESHOLD, 5.0218)   # fungsi kurang
stopping_crit = (Stop.TRESHOLD, 16.49)  # fungsi eksponen
# stopping_crit = (gea.Stop.NO_IMPROVE, round(maks_generasi*.2))

In [ ]:
# %% single run
gen = Gea(fungsi_fitness=fungsiFitness,
              ranges=((-1, 2), (-1, 1)),
              resolusi=resolusi,
              ukuran_populasi=ukuran_populasi)

fenotip, iterasi, fitness_history = gen.fit(stopping_crit=stopping_crit,
                                            maks_generasi=maks_generasi,
                                            crossover_rate=crossover_rate,
                                            peluang_mutasi=peluang_mutasi,
                                            rekam_history=rekam_history,
                                            verbose=verbosity_level)

print('Banyak generasi:', iterasi)
print('Kromosom:\n', gen.best_individu[0].kromosom)
print('fitness =', fungsiFitness(fenotip))
print('h(x1, x2) =', h(fenotip))
print('x1 =', fenotip[0])
print('x2 =', fenotip[1])

if rekam_history:
  plt.title('Fitness History')
  plt.xlabel('Generasi')
  plt.ylabel('Fitness')
  plt.plot(fitness_history)
else:
  print('Tidak ada rekaman')

In [ ]:
# %% Statistik
num_of_run = 20

best_fit_hist = []
iterasi_hist = []

gen = Gea(fungsi_fitness=fungsiFitness,
              ranges=((-1, 2), (-1, 1)),
              resolusi=resolusi,
              ukuran_populasi=ukuran_populasi)

print('Running', end='')
for _ in range(num_of_run):
  gen.reset()
  fenotip, iterasi, _ = gen.fit(stopping_crit=stopping_crit,
                              maks_generasi=maks_generasi,
                              crossover_rate=crossover_rate,
                              peluang_mutasi=peluang_mutasi,
                              rekam_history=rekam_history,
                              verbose=0)
  best_fit_hist.append(fungsiFitness(fenotip))
  iterasi_hist.append(iterasi)
  print('.', end='')
print(' done!')

print('Rata-rata banyak generasi:', sum(iterasi_hist)/num_of_run)
print('Rata-rata fitness akhir:', sum(best_fit_hist)/num_of_run)

plt.subplot(1,2,1)
plt.title('Histogram Fitness')
plt.xlabel('fitness')
plt.ylabel('frekuensi')
_ = plt.hist(best_fit_hist)

plt.tight_layout(pad=2.5)
plt.subplot(1,2,2)
plt.title('Histogram #Generasi')
plt.xlabel('banyak generasi')
_ = plt.hist(iterasi_hist)

## GA Tuner

Berfungsi untuk menemukan parameter yang tepat untuk kasus GA di main.py

(tidak disarankan untuk dijalankan karena membutuhkan resource yang cukup besar)

### tuner.py

In [ ]:
# ========= Tuner genetika =========
# untuk mencari parameter algoritma genetika
# cukup digunakan sekali untuk memperoleh parameter
# genetika yang dibutuhkan.
#
# ditulis oleh Jordi Yaputra (130110353)

""" ============= Segel =============
# [dapat buka dengan cara berikan '#' di baris 8]
#
# Untuk mencegah file ini dijalankan tanpa sengaja karena
# dapat memakan waktu dan resources yang cukup besar.

# %% import
import time

# %% hipothesis dan fitness
def tuner_h(fenotip):
  num_run = 3

  uk_pop, res, p_mut, x_rate = fenotip

  gen = Gea(fungsi_fitness=fungsiFitness,
                ranges=((-1, 2), (-1, 1)),
                resolusi=round(res),
                ukuran_populasi=round(uk_pop))

  maks_generasi = int((10000 - uk_pop * (1 - x_rate)) \
                   / (uk_pop * x_rate))

  sum_fit = 0
  sum_it = 0


  for i in range(num_run):
    gen.reset()
    fen, it, _ = gen.fit(stopping_crit=stopping_crit,
                          maks_generasi=maks_generasi,
                          crossover_rate=x_rate,
                          peluang_mutasi=p_mut,
                          rekam_history=False,
                          verbose=0)
    sum_fit += fungsiFitness(fen)
    sum_it += it

  mean_fit = sum_fit/num_run
  mean_it = sum_it/num_run

  return (mean_fit, mean_it)

def tuner_fit_func(fenotip):
  mean_fit, mean_it = tuner_h(fenotip)
  return mean_fit/17 * 1/mean_it

# %% Skala yang perlu diatur
ranges = (
  (50, 800),  # ukuran populasi
  (4, 10),    # resolusi
  (.01, .2),  # peluang_mutasi
  (.1, 1)     # crossover rate
)

# %% main
tuner = Gea(fungsi_fitness=tuner_fit_func,
              ranges=ranges,
              resolusi=8,
              ukuran_populasi=20)

start_time = time.time()

fen, _, rekaman = tuner.fit(maks_generasi=26,
                            peluang_mutasi=.05,
                            crossover_rate=.45,
                            rekam_history=True,
                            verbose=1)

eta = time.time() - start_time
print('Elapsed time: %.2f s', eta)

print(fen)
print(tuner_h(fen))

plt.plot(rekaman)

# """